In [2]:
import pandas as pd

In [3]:
skincare=pd.read_csv('SkinCareDetail.csv')
makeup=pd.read_csv('MakeupDetail.csv')
skincare.shape,makeup.shape

((4397, 11), (4896, 11))

In [17]:
#Combine makeup and skincare data
skincare.insert(8,'Main_Category',["Skincare"]*4397,True)
makeup.insert(8,'Main_Category',["Makeup"]*4896,True)
ulta=pd.concat([skincare,makeup],axis=0)
ulta.index=range(0,len(ulta))
ulta['Price']=[x.split("Original Price")[0] for x in ulta['Price']]
ulta.sample(3)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Main_Category,Main_Category,Category,Details,Image
5696,e.l.f. Cosmetics,Pro Eyelash Curler,3.3,$5.00,https://www.ulta.com/p/pro-eyelash-curler-pimp...,2560974,223.0,Ingredients are not available,Makeup,Makeup,Makeup,Makeup Brushes & Tools,Details\nBenefits\nPremium eyelash curler\nPro...,https://media.ulta.com/i/ulta/1299?w=582&h=582...
769,Dermalogica,Intensive Moisture Cleanser,4.6,$42.00 - $66.00,https://www.ulta.com/p/intensive-moisture-clea...,2535737,1763.0,"Water/Aqua, Glycerin Caprylic / Capric Triglyc...",Skincare,Skincare,Skincare,Cleansers,"Details\nBenefits\nRemoves make-up, sunscreen,...",https://media.ulta.com/i/ulta/2535737?w=156&h=156
5010,Lancôme,Artliner Precision Felt Tip Liquid Eyeliner,4.4,$31.00,https://www.ulta.com/p/artliner-precision-felt...,2301337,1001.0,Ingredients are not available,Makeup,Makeup,Makeup,Eyes,Details\nFeatures\nLancôme Artliner Precision ...,https://media.ulta.com/i/ulta/613?w=582&h=582&...


In [24]:
indexAge=ulta[ ulta['Category'].isin(['Makeup Brushes & Tools' ,'Makeup Bags & Organizers','Skincare Tools',
                                     'Bath & Body Accessories'
                                     ])].index
ulta.drop(indexAge,inplace=True)

In [25]:
#Drop lines without ingredients provided
indexAge=ulta[ ulta['Ingredient'].isin(['Ingredients are not available' ,'Ingredients is not available',
                                        'See full ingredient listing on packaging.',
                                        'See packaging for full ingredient listing.',
                                        'See packaging for full list of ingredients.',
                                        'See individual packaging for a full list of ingredients.',
                                        'A complete listing of ingredients are available on individual product pages.'
                                       ])].index
ulta.drop(indexAge,inplace=True)

In [26]:
#Drop duplicates

ulta.drop_duplicates(subset=["ID"],inplace=True)
ulta.to_csv('Clean_data',index=None)